In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import random

# 데이터프레임을 다루기 위한 pandas import
import pandas as pd

# 크롷링하는 함수
def croll():
    for i in range(1, 22):
        if i == 6:
            continue
        try:
            driver.find_element(By.XPATH, f'/html/body/main/article[2]/section/section[2]/section[2]/article[{i}]/section[1]/section[2]/article/a/h2').click()
        except:
            break
        
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)
        
        try:
            title = driver.find_element(By.XPATH, '//*[@id="thesisTitle"]').text
        except:
            title = ''

        try:
            date = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__info > section.thesisDetail__journal > ul > li:nth-child(4) > span:nth-child(1)').text
        except:
            try:
                date = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__info > section.thesisDetail__journal.projectDetail__journal > ul > li:nth-child(2) > p.projectDetail__advisoir__desc').text
            except:
                date = ''
        
        try:
            abstract = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__abstract > div.abstractTxt').text
        except:
            abstract = ''

        try:
            # 모든 키워드 <a> 태그 찾기
            keyword_elements = driver.find_elements(By.CSS_SELECTOR, 'div.keywordWrap a.keywordWrap__keyword')
            
            # 각 키워드의 텍스트를 가져와 문자열로 결합
            keyword_text = ", ".join([element.text for element in keyword_elements])
        except:
            keyword_text = ''
        
        titles.append(title)
        dates.append(date)
        abstracts.append(abstract)
        keywords.append(keyword_text)
        
        time.sleep(1.5)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1.5)
        if i % 3 == 0:
            time.sleep(random.randint(3,7))


# 논문 검색 설정
search = input('검색할 키워드 : ').strip()
min_year = input('추출할 최소 년도를 입력 : ').strip()
max_year = input('추출할 최대 년도를 입력 : ')
time.sleep(1)

# 크롬드라이버 및 메인페이지 실행
driver = webdriver.Chrome()
driver.get('https://www.dbpia.co.kr/')

# 페이지가 완전히 로딩되도록 5초동안 기다림
time.sleep(5)

# 설정한 키워드로 검색
driver.find_element(By.XPATH, '//*[@id="searchInput"]').click()
time.sleep(0.5)
driver.find_element(By.XPATH, '//*[@id="searchInput"]').send_keys(search)
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="submitSearchInput"]').click()
time.sleep(3)

# 페이지를 100개씩 보기로 설정
# element = driver.find_element(By.XPATH, '//*[@id="selectWrapper"]')
# driver.execute_script("arguments[0].click();", element)
# time.sleep(3)
# driver.find_element(By.XPATH, '//*[@id="get100PerPage"]')
# driver.execute_script("arguments[0].click();", element)
# time.sleep(3)

# 페이지를 불러올 년도 설정
driver.find_element(By.XPATH, '//*[@id="yearRangeMin"]').click()
driver.find_element(By.XPATH, '//*[@id="yearRangeMin"]').send_keys(min_year)
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="yearRangeMax"]').click()
driver.find_element(By.XPATH, '//*[@id="yearRangeMax"]').send_keys(max_year)
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="yearRangeSearch"]').click()
time.sleep(3)

# 페이지 수 게산
article_num = driver.find_element(By.XPATH, '//*[@id="totalCount"]').text
article_num = int(article_num[:-1].replace(',',''))
page_num = article_num // 20 + 1
print('추출할 논문의 수 : ', article_num)

# 정보를 저장해둘 리스트 선언
titles = []
dates = []
abstracts = []
keywords = []

# 첫 페이지 크롤링
croll()

# 나머지 페이지 크롤링
for x in range(2,page_num+1):
    try:
        if x % 10 == 1:
            driver.find_element(By.XPATH, '//*[@id="goNextPage"]').click()
        else:
            if x % 10 ==0:
                driver.find_element(By.XPATH, '//*[@id="pageList"]/a[10]').click()
            else:
                driver.find_element(By.XPATH, f'//*[@id="pageList"]/a[{x%10}]').click()
        time.sleep(random.randint(5,7))
        if x % 5 == 0:
            time.sleep(random.randint(10, 15))
        croll()
    except:
        break

# 데이터 프레임 생성
data = pd.DataFrame({
    'title' : titles,
    'date' : dates,
    'abstract' : abstracts,
    'keyword' : keywords
})

# 키워드 데이터를 키워드용 데이터프레임으로 변환
keywords_data = pd.DataFrame({
    'keywords': keywords  # 각 논문별 키워드 리스트
})

# 키워드 리스트를 explode로 펼치기
keywords_data = keywords_data.explode('keywords').reset_index(drop=True)
    

# 데이터 프레임을 csv 형태로 저장
csv_file_path = f"{search}_{min_year}_dbpia.csv"
data.to_csv(csv_file_path, index=False)

# 데이터 프레임을 csv 형태로 저장
csv_file_path_keywords = csv_file_path = f"{search}_{min_year}_dbpia_keywords.csv"
keywords_data.to_csv(csv_file_path_keywords, index=False)

driver.close()

print('추출한 논문의 수 : ', len(titles))

#데이터 확인
dff = pd.read_csv(csv_file_path, encoding='utf-8')
dff.info()

dff.head()


추출할 논문의 수 :  1562
추출한 논문의 수 :  726
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keywords  398 non-null    object
dtypes: object(1)
memory usage: 5.8+ KB


,keywords
0,"#CNN, #DL Model, #Training and Validation Data..."
1,"#바퀴식 주행로봇, #경로 찾기, #딥강화학습, #딥러닝, #자율주행로봇, #Whe..."
2,"#분류, #감성어휘, #병원, #실내이미지, #딥러닝, #Classification..."
3,"#Instance segmentation, #Deep learning, #Slope..."
4,"#PM\n10\nDeep Learning, #Satellite Image, #Sen..."


In [ ]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import random

# 데이터프레임을 다루기 위한 pandas import
import pandas as pd

# 크롷링하는 함수
def croll():
    for i in range(1, 22):
        if i == 6:
            continue
        try:
            driver.find_element(By.XPATH, f'/html/body/main/article[2]/section/section[2]/section[2]/article[{i}]/section[1]/section[2]/article/a/h2').click()
        except:
            break
        
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)
        
        try:
            title = driver.find_element(By.XPATH, '//*[@id="thesisTitle"]').text
        except:
            title = ''

        try:
            date = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__info > section.thesisDetail__journal > ul > li:nth-child(4) > span:nth-child(1)').text
        except:
            try:
                date = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__info > section.thesisDetail__journal.projectDetail__journal > ul > li:nth-child(2) > p.projectDetail__advisoir__desc').text
            except:
                date = ''
        
        try:
            abstract = driver.find_element(By.CSS_SELECTOR, '#dpMain > section > section.thesisDetail__abstract > div.abstractTxt').text
        except:
            abstract = ''

        try:
            # 모든 키워드 <a> 태그 찾기
            keyword_elements = driver.find_elements(By.CSS_SELECTOR, 'div.keywordWrap a.keywordWrap__keyword')
            
            # 각 키워드의 텍스트를 가져와 문자열로 결합
            keyword_text = ", ".join([element.text for element in keyword_elements])
        except:
            keyword_text = ''
        
        titles.append(title)
        dates.append(date)
        abstracts.append(abstract)
        keywords.append(keyword_text)
        
        time.sleep(1.5)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1.5)
        if i % 3 == 0:
            time.sleep(random.randint(3,7))


# 논문 검색 설정
search = input('검색할 키워드 : ').strip()
min_year = input('추출할 최소 년도를 입력 : ').strip()
max_year = input('추출할 최대 년도를 입력 : ')
time.sleep(1)

# 크롬드라이버 및 메인페이지 실행
driver = webdriver.Chrome()
driver.get('https://www.dbpia.co.kr/')

# 페이지가 완전히 로딩되도록 5초동안 기다림
time.sleep(5)

# 설정한 키워드로 검색
driver.find_element(By.XPATH, '//*[@id="searchInput"]').click()
time.sleep(0.5)
driver.find_element(By.XPATH, '//*[@id="searchInput"]').send_keys(search)
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="submitSearchInput"]').click()
time.sleep(3)

# 페이지를 100개씩 보기로 설정
# element = driver.find_element(By.XPATH, '//*[@id="selectWrapper"]')
# driver.execute_script("arguments[0].click();", element)
# time.sleep(3)
# driver.find_element(By.XPATH, '//*[@id="get100PerPage"]')
# driver.execute_script("arguments[0].click();", element)
# time.sleep(3)

# 페이지를 불러올 년도 설정
driver.find_element(By.XPATH, '//*[@id="yearRangeMin"]').click()
driver.find_element(By.XPATH, '//*[@id="yearRangeMin"]').send_keys(min_year)
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="yearRangeMax"]').click()
driver.find_element(By.XPATH, '//*[@id="yearRangeMax"]').send_keys(max_year)
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="yearRangeSearch"]').click()
time.sleep(3)

# 페이지 수 게산
article_num = driver.find_element(By.XPATH, '//*[@id="totalCount"]').text
article_num = int(article_num[:-1].replace(',',''))
page_num = article_num // 20 + 1
print('추출할 논문의 수 : ', article_num)

# 정보를 저장해둘 리스트 선언
titles = []
dates = []
abstracts = []
keywords = []

# 첫 페이지 크롤링
croll()

# 나머지 페이지 크롤링
for x in range(2,page_num+1):
    try:
        if x % 10 == 1:
            driver.find_element(By.XPATH, '//*[@id="goNextPage"]').click()
        else:
            if x % 10 ==0:
                driver.find_element(By.XPATH, '//*[@id="pageList"]/a[10]').click()
            else:
                driver.find_element(By.XPATH, f'//*[@id="pageList"]/a[{x%10}]').click()
        time.sleep(random.randint(5,7))
        if x % 5 == 0:
            time.sleep(random.randint(10, 15))
        croll()
    except:
        break

# 데이터 프레임 생성
data = pd.DataFrame({
    'title' : titles,
    'date' : dates,
    'abstract' : abstracts,
    'keyword' : keywords
})

# 키워드 데이터를 키워드용 데이터프레임으로 변환
keywords_data = pd.DataFrame({
    'keywords': keywords  # 각 논문별 키워드 리스트
})

# 키워드 리스트를 explode로 펼치기
keywords_data = keywords_data.explode('keywords').reset_index(drop=True)
    

# 데이터 프레임을 csv 형태로 저장
csv_file_path = f"{search}_{min_year}_dbpia.csv"
data.to_csv(csv_file_path, index=False)

# 데이터 프레임을 csv 형태로 저장
csv_file_path_keywords = csv_file_path = f"{search}_{min_year}_dbpia_keywords.csv"
keywords_data.to_csv(csv_file_path_keywords, index=False)

driver.close()

print('추출한 논문의 수 : ', len(titles))

#데이터 확인
dff = pd.read_csv(csv_file_path, encoding='utf-8')
dff.info()

dff.head()
